In [1]:
# Parameters
params = {
    # Basic network / model settings
    "network_type": "BA",      # "random" or "BA"
    "n_nodes": 120,
    "p": 0.05,                 # ER probability (ignored for BA)
    "m": 2,                    # BA m parameter (edges per new node)

    # Payoff & infrastructure
    "ratio": 2.3,              # if provided, a0 is computed from ratio: a0 = ratio*b - beta_I*I0
    "a0": 2.0,                 # base coordination payoff (used if ratio is None)
    "beta_I": 2.0,
    "b": 1.0,
    "I0": 0.05,
    "g_I": 0.05,

    # Simulation control
    "T": 200,
    "n_trials": 20,            # used for intervention collections: reduce for quick runs
    "batch_size": 8,           # for sweeps
    "init_noise_I": 0.04,

    # Strategy & sampling
    "strategy_choice_func": "imitate",   # "imitate" or "logit"
    "tau": 1.0,

    # Intervention / policy (example)
    "subsidy": {"start": 10, "end": 60, "delta_a0": 0.4, "delta_beta_I": 0.0},

    # Parallelism/backends
    "max_workers": 1,
    "backend": "thread",      # "thread" or "process" (used by phase sweep helper)

    # File outputs
    "plots_dir": "plots",     # directory (created automatically)
}
# Print params for confirmation
import pprint
pprint.pprint(params)


{'I0': 0.05,
 'T': 200,
 'a0': 2.0,
 'b': 1.0,
 'backend': 'thread',
 'batch_size': 8,
 'beta_I': 2.0,
 'g_I': 0.05,
 'init_noise_I': 0.04,
 'm': 2,
 'max_workers': 1,
 'n_nodes': 120,
 'n_trials': 20,
 'network_type': 'BA',
 'p': 0.05,
 'plots_dir': 'plots',
 'ratio': 2.3,
 'strategy_choice_func': 'imitate',
 'subsidy': {'delta_a0': 0.4, 'delta_beta_I': 0.0, 'end': 60, 'start': 10},
 'tau': 1.0}


In [2]:

# Ensure current directory is on sys.path so local modules are importable
import sys, os
sys.path.insert(0, os.getcwd())

# Standard imports
import numpy as np
import pandas as pd

# Import the provided modules (assumes they are in the same directory)
from ev_core import EVStagHuntModel, set_initial_adopters
from ev_experiments import (
    run_timeseries_trial,
    collect_intervention_trials,
    traces_to_long_df,
    ratio_sweep_df,
    phase_sweep_df,
    run_timeseries_trial,
)
from ev_plotting import (
    plot_fanchart,
    plot_spaghetti,
    plot_density,
    plot_ratio_sweep,
    plot_phase_plot,
)

# Create plots directory if needed
os.makedirs(params.get("plots_dir", "plots"), exist_ok=True)

print('Modules imported successfully.')


Modules imported successfully.


In [3]:

# Quick intervention example (small numbers so it runs fast)
# Use params defined above. Edit params and re-run this cell to experiment.
T = params["T"]
n_trials = max(2, int(min(params["n_trials"], 30)))  # cap for quick demo
scenario_kwargs = {
    "ratio": params.get("ratio"),
    "a0": params.get("a0"),
    "beta_I": params.get("beta_I"),
    "b": params.get("b"),
    "g_I": params.get("g_I"),
    "I0": params.get("I0"),
    "network_type": params.get("network_type"),
    "n_nodes": params.get("n_nodes"),
    "p": params.get("p"),
    "m": params.get("m"),
    "collect": True,
    "X0_frac": 0.40,
    "init_method": "random",
}

subsidy_params = params.get("subsidy", None)

print('Running', n_trials, 'trials (baseline and subsidy) with T=', T)
baseline_X, baseline_I, subsidy_X, subsidy_I, baseline_df, subsidy_df = collect_intervention_trials(
    n_trials=n_trials,
    T=T,
    scenario_kwargs=scenario_kwargs,
    subsidy_params=subsidy_params,
    max_workers=params.get("max_workers", 1),
    seed_base=100,
    strategy_choice_func=params.get("strategy_choice_func", "imitate"),
    tau=params.get("tau", 1.0),
)

print("Baseline DF shape:", baseline_df.shape)
print("Subsidy  DF shape:", subsidy_df.shape)
print("Baseline final X_mean:", float(baseline_df["X_mean"].iloc[-1]))
print("Subsidy  final X_mean:", float(subsidy_df["X_mean"].iloc[-1]))

# Save fanchart via plotting utilities (plot_fanchart expects traces_df)
traces_df = traces_to_long_df(baseline_X, subsidy_X)
fanchart_path = plot_fanchart(traces_df, out_path=os.path.join(params.get("plots_dir","plots"), "ev_intervention_fanchart_demo.png"))
print("Saved fanchart to:", fanchart_path)


Running 20 trials (baseline and subsidy) with T= 200
Baseline DF shape: (200, 6)
Subsidy  DF shape: (200, 6)
Baseline final X_mean: 0.4445833333333333
Subsidy  final X_mean: 0.5258333333333333
Saved fanchart to: plots\ev_intervention_fanchart_demo.png


In [4]:

# Ratio sweep (quick demo)
X0_frac = 0.40
ratio_values = np.linspace(0.8, 3.5, 9)  # small for demo; increase for publication-quality
sweep_df = ratio_sweep_df(
    X0_frac=X0_frac,
    ratio_values=ratio_values,
    scenario_kwargs={
        "beta_I": params["beta_I"],
        "b": params["b"],
        "g_I": params["g_I"],
        "I0": params["I0"],
        "network_type": params["network_type"],
        "n_nodes": params["n_nodes"],
        "p": params["p"],
        "m": params["m"],
    },
    T=100,
    batch_size=4,
    init_noise_I=params.get("init_noise_I", 0.04),
    strategy_choice_func=params.get("strategy_choice_func", "logit"),
    tau=params.get("tau", 1.0),
)
print('Ratio sweep computed, head:')
display(sweep_df.head())
ratio_plot_path = plot_ratio_sweep(sweep_df, out_path=os.path.join(params.get("plots_dir","plots"), "ev_ratio_sweep_demo.png"))
print('Saved ratio sweep plot to:', ratio_plot_path)


Ratio sweep computed, head:


,ratio,X_mean
0,0.8000,0.000000
1,1.1375,0.000000
2,1.4750,0.500000
3,1.8125,0.750000
4,2.1500,0.289583


Saved ratio sweep plot to: plots\ev_ratio_sweep_demo.png


In [5]:

# Phase plot (coarse, smaller grid for demo speed)
X0_values = np.linspace(0.0, 1.0, 11)
ratio_values = np.linspace(0.8, 3.5, 11)
phase_df = phase_sweep_df(
    max_workers=1,
    backend="thread",
    X0_values=X0_values,
    ratio_values=ratio_values,
    scenario_kwargs={
        "I0": params["I0"],
        "beta_I": params["beta_I"],
        "b": params["b"],
        "g_I": params["g_I"],
        "network_type": params["network_type"],
        "n_nodes": params["n_nodes"],
        "p": params["p"],
        "m": params["m"],
    },
    batch_size=4,
    init_noise_I=params.get("init_noise_I", 0.04),
    T=100,
    strategy_choice_func=params.get("strategy_choice_func", "logit"),
    tau=params.get("tau", 1.0),
)
print('Phase sweep computed: rows =', len(phase_df))
phase_path = plot_phase_plot(phase_df, out_path=os.path.join(params.get("plots_dir","plots"), "ev_phase_plot_demo.png"))
print('Saved phase plot to:', phase_path)


Phase sweep computed: rows = 121
Saved phase plot to: plots\ev_phase_plot_demo.png
